In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
from PIL import Image
import cv2
import numpy as np
import onnxruntime as ort


In [5]:
class VehicleCount:
    def __init__(self):
        self.cuda = False
        self.w = "/content/drive/My Drive/Adaptive-Traffic-Light-YOLOV7/yolov7-tiny.onnx"
        self.session = ort.InferenceSession(self.w)

    def padding(self, im,new_shape=(640,640),color=(114,114,114),auto=True,scaleup=True,stride=32):
        shape = im.shape[:2]
        r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
        new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
        dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
        if auto:
            dw, dh = np.mod(dw, stride), np.mod(dh, stride)
        dw /= 2
        dh /= 2
        if shape[::-1] != new_unpad:
            im = cv2.resize(im, new_unpad)
        top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
        left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
        im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        return im, r, (dw, dh)

    def getVehcileCount(self,img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = img.copy()
        image, ratio, dwdh = self.padding(image, auto=False)
        image = image.transpose((2, 0, 1))
        image = np.expand_dims(image, 0)
        im = image.astype(np.float32)
        im /= 255
        outname = [i.name for i in self.session.get_outputs()]
        inname = [i.name for i in self.session.get_inputs()]
        inp = {inname[0]:im}
        outputs = self.session.run(outname, inp)[0]
        ori_images = [img.copy()]
        for i, (batch_id, x0, y0, x1, y1, cls_id, score) in enumerate(outputs):
            image = ori_images[int(batch_id)]
            box = np.array([x0, y0, x1, y1])
            box -= np.array(dwdh*2)
            box /= ratio
            box = box.round().astype(np.int32).tolist()
            color = (255,255,0)
            cv2.rectangle(image,box[:2],box[2:],color,2)
        img2 = Image.fromarray(ori_images[0])
        return len(outputs),img2

In [8]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst


def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


obj = VehicleCount()
lane = []


imgLane1 = cv2.imread('/content/drive/My Drive/Adaptive-Traffic-Light-YOLOV7/Resources/b5.jpeg')
imgLane2 = cv2.imread('/content/drive/My Drive/Adaptive-Traffic-Light-YOLOV7/Resources/b3.jpeg')

# imgLane1 = cv2.imread('/content/drive/My Drive/Adaptive-Traffic-Light-YOLOV7/Resources/b1.jpeg')
# imgLane2 = cv2.imread('/content/drive/My Drive/Adaptive-Traffic-Light-YOLOV7/Resources/b4.jpeg')

countLane1, boxLane1 = obj.getVehcileCount(imgLane1)
lane.append(countLane1)
# boxLane1.show()


countLane2, boxLane2 = obj.getVehcileCount(imgLane2)
lane.append(countLane2)
# boxLane2.show()

imgLane3 = cv2.imread('/content/drive/My Drive/Adaptive-Traffic-Light-YOLOV7/Resources/b2.jpeg')
countLane3, boxLane3 = obj.getVehcileCount(imgLane3)
lane.append(countLane3)
# boxLane3.show()

imgLane4 = cv2.imread('/content/drive/My Drive/Adaptive-Traffic-Light-YOLOV7/Resources/b6.jpeg')
countLane4, boxLane4 = obj.getVehcileCount(imgLane4)
lane.append(countLane4)
# boxLane4.show()
img1 = get_concat_h(boxLane1, boxLane2)
img2 = get_concat_h(boxLane3, boxLane4)

# img1.show()
final_img = get_concat_v(img1, img2)
final_img.show()
print("-"*20)
print("\tLane Count")
print("-"*20)
print(f'Lane 1 - {countLane1}')
print(f'Lane 2 - {countLane2}')
print(f'Lane 3 - {countLane3}')
print(f'Lane 4 - {countLane4}')
print("-"*20)
lessGreenTime = 15
normalGreenTime = 30
heavyGreenTime = 50
totalTime = 0
laneGreenTime = []
laneRedTime = []
laneF = []
for i in range(0, 4):
    if lane[i] < 3:
        laneGreenTime.append(lessGreenTime)
        totalTime += lessGreenTime
        laneF.append('L')
    elif lane[i] < 10:
        laneGreenTime.append(normalGreenTime)
        totalTime += normalGreenTime
        laneF.append('N')
    else:
        laneGreenTime.append(heavyGreenTime)
        totalTime += heavyGreenTime
        laneF.append('H')
for i in laneGreenTime:
    laneRedTime.append(totalTime-i)
print("\n\n")
print("-"*53)
print("\t\t\t\tTime Allocations\t")
print("-"*53)
for i in range(0, 4):
    print(f'Lane {i+1}\t| {laneF[i]}\t|\tGreen Time - {laneGreenTime[i]}\t|\tRed Time - {laneRedTime[i]}\t|')
print("-"*53)

--------------------
	Lane Count
--------------------
Lane 1 - 7
Lane 2 - 8
Lane 3 - 7
Lane 4 - 5
--------------------



-----------------------------------------------------
				Time Allocations	
-----------------------------------------------------
Lane 1	| N	|	Green Time - 30	|	Red Time - 90	|
Lane 2	| N	|	Green Time - 30	|	Red Time - 90	|
Lane 3	| N	|	Green Time - 30	|	Red Time - 90	|
Lane 4	| N	|	Green Time - 30	|	Red Time - 90	|
-----------------------------------------------------


In [9]:
final_img

Output hidden; open in https://colab.research.google.com to view.